In [15]:
using Plots
using LinearAlgebra
using DifferentialEquations
plotly()

Plots.PlotlyBackend()

In [16]:
include("../src/SailorMoon.jl")   # relative path to main file of module
include("../../julia-r3bp/R3BP/src/R3BP.jl")

Main.R3BP

In [17]:
param3b = SailorMoon.dyanmics_parameters()

Main.SailorMoon.dynamics_params(0.987849414390376, 0.01215058560962404, 328900.5598102475, 384748.32292972936, 375700.3437894195, 388.8212386592645, -0.9251999994040079, 0.9251999994040079, 0.01709689063726318, 7.601281331451572)

In [58]:
τ = 0.1
tmax = 0.4  # N = kg m / s^2
mdot = tmax / 9.8 / 3500  # kg/s -> Isp = 3500s

# # normalize
mstar = 5.972e24 / param3b.mu1 
tmax = 1e-2  #tmax / mstar / param3b.lstar * (param3b.tstar)^2 
mdot = 1e-3  #mdot / mstar * param3b.tstar 
mf = 1  #3000 / mstar  

n = 1
SCParam = [mdot, tmax, mf]

3-element Vector{Float64}:
 0.001
 0.01
 1.0

In [134]:
param3b.mus

328900.5598102475

In [82]:
# set up LPOArr 

lp = 2
Az_km = 28000.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="z")
res.flag

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix

ys0 = R3BP.get_eigenvector(monodromy, true, 1)
# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12
);

Halo guess Az_km: 28000.0
Linear stability ν = 290.9383214356218


In [135]:
# extract optimization var
ϕ = 0.0
θm = deg2rad(120)

tof      = (90 * 86400) / param3b.tstar
c_launch = [mf*1.2, 3.0, 0.0, 0.0]  # m0, dv1, long, lat
c_arr    = [ϕ, θm]
tau1     = [1, 0.0, 0.0]  # discretization numbers are the same for the first & second arc
tau2     = [1, 0.0, 0.0]

# make a decision vector x 
x = []
append!(x,tof)
append!(x,c_launch)
append!(x,c_arr)
append!(x,tau1)
append!(x,tau2)
println(x)

Any[20.697345979428906, 1.2, 3.0, 0.0, 0.0, 0.0, 2.0943951023931953, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]


In [136]:
param3b.lstar/param3b.tstar

1.0240829674230516

In [137]:
# set up the default ODE
state0 = zeros(7)
tspan = zeros(2)
params = zeros(10)

Propagator = SailorMoon.ODEPropagator(
    Tsit5(), 1e-12, 1e-12, ODEProblem(SailorMoon.rhs_bcr4bp_with_mass!, state0, tspan, params)
);

In [138]:
# propagate
res, state_fwd, state_bkwd, sol_fwd, sol_bkwd = SailorMoon.sf_propagate(x, n, SCParam, Propagator, param3b, LPOArrival);
print(res)

[-9.095691148386225, -32.14570327688327, 1.102088798285221, -32.697381780155965, 9.6921067755877, -0.09053764171068522, -0.17930265402056889]

In [139]:
Array(sol_fwd[1])

7×1372 Matrix{Float64}:
  0.0197956   0.0197952   0.0197946  …   18.5577    18.663     18.6957
 -0.0118471  -0.0116512  -0.0115621       7.54374    7.33447    7.26813
  0.0         0.0         0.0             0.0        0.0        0.0
  0.0109609  -0.0511087  -0.07936         8.01092    7.8051     7.73982
 10.6038     10.6034     10.603         -15.6624   -15.7746   -15.8094
  0.0         0.0         0.0        …    0.0        0.0        0.0
  1.2         1.2         1.2             1.18967    1.18966    1.18965

In [140]:
Array(sol_bkwd[1])

7×948 Matrix{Float64}:
 -0.544182   -0.542702   -0.541187   -0.53925    …    9.45173      9.59998
  0.942551    0.943409    0.944294    0.945435      -24.9134     -24.8776
 -0.0606218  -0.0605398  -0.0604546  -0.0603437       1.10155      1.10209
 -1.10206    -1.09777    -1.09335    -1.08766       -24.9869     -24.9576
 -0.636275   -0.638669   -0.641113   -0.64423        -5.96769     -6.11729
 -0.0606224  -0.0612957  -0.0619872  -0.0628745  …   -0.0905772   -0.0905376
  1.0         1.0         1.0         1.0             1.01034      1.01035

In [149]:
thetas = LinRange(0, 2π, 100)
xs_earth_circle = [-param3b.mu2*cos(theta) for theta in thetas];
ys_earth_circle = [-param3b.mu2*sin(theta) for theta in thetas];
xs_moon_circle = [(1-param3b.mu2)*cos(theta) for theta in thetas];
ys_moon_circle = [(1-param3b.mu2)*sin(theta) for theta in thetas];

In [153]:
ptraj = plot(frame_style=:box, size=(700,700), aspect_ratio=:equal)
plot!(ptraj, Array(sol_fwd[1])[1,:], Array(sol_fwd[1])[2,:])
plot!(ptraj, Array(sol_bkwd[1])[1,:], Array(sol_bkwd[1])[2,:])

plot!(ptraj, xs_moon_circle, ys_moon_circle, color=:black, linestyle=:solid, label="Moon trajectory", linewidth=0.4)

ptraj